In [18]:
# Config for notebook - optional
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import pandas as pd
pd.set_option('display.max_colwidth',100)

In [17]:
# Delta lake docs
display(HTML("""<a href="https://docs.delta.io/latest/delta-intro.html">Delta Lake Docs</a> """))

In [37]:
import pyspark
import pyspark.sql.functions as F

# Instantiate spark session with delta lake
spark = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

from delta.tables import *

In [7]:
# Read in data in to dataframe

df = spark.read.option("header",True).csv('../data/train.csv')

df.count()

df.show(5)

913000

+----------+-----+----+-----+
|      date|store|item|sales|
+----------+-----+----+-----+
|2013-01-01|    1|   1|   13|
|2013-01-02|    1|   1|   11|
|2013-01-03|    1|   1|   14|
|2013-01-04|    1|   1|   13|
|2013-01-05|    1|   1|   10|
+----------+-----+----+-----+
only showing top 5 rows



In [12]:
# Write as delta table.
df.write.format("delta").save("../lake")

In [9]:
# Restart kernel and readin data from delta lake

delta_df = spark.read.format("delta").load("../lake")

type(delta_df)

delta_df.count()

pyspark.sql.dataframe.DataFrame

913000

In [20]:
# Create a new data frame with a new column
new_df = df.withColumn('col_n', df.store + df.item)

new_df.show()

In [26]:
# Write this back to delta lake. Note: without overwrite and mergescehma we will get an error as Delta lake wont allow us to overwrite data with new column

new_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("../lake")

In [20]:
# Lets check Delta table's meta data
deltaTable = DeltaTable.forPath(spark, "../lake")

In [21]:
# As we can see below, Delta lake has history of our initial write and then changes made subsequently.
deltaTable.history().toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,1,2021-04-07 12:40:19.961,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,None,False,"{'numOutputRows': '913000', 'numOutputBytes': '1776421', 'numFiles': '4'}",None
1,0,2021-04-07 12:22:02.304,None,None,WRITE,"{'mode': 'ErrorIfExists', 'partitionBy': '[]'}",None,None,None,NaN,None,True,"{'numOutputRows': '913000', 'numOutputBytes': '1773875', 'numFiles': '4'}",None


In [12]:
# Below are all parquet files related to above changes
!ls ../lake

_delta_log
part-00000-457fb9cf-fd52-423d-9af9-c2922d0c85fe-c000.snappy.parquet
part-00000-bb692ea7-87a1-4af0-b7f3-95189f10bdcf-c000.snappy.parquet
part-00001-225c2da3-6d10-4561-a7eb-66d51c4e0720-c000.snappy.parquet
part-00001-a6373976-cac8-45d4-ab3d-120c40bd7222-c000.snappy.parquet
part-00002-2236d358-9a8e-4d96-bb24-38caecae18d6-c000.snappy.parquet
part-00002-712810b1-3c68-4e17-9536-246c20c10d4b-c000.snappy.parquet
part-00003-376097a8-e45b-4d0e-a3fd-7940325b6c65-c000.snappy.parquet
part-00003-7557dfdc-4c47-42d3-9801-4c80cafa6c21-c000.snappy.parquet


In [30]:
# Restart kernel and now we can read data as of version we are interested in.
old_table = spark.read.format("delta").option("versionAsOf", 0).load("../lake")
print((old_table.count(), len(old_table.columns)))
old_table.limit(5).toPandas()

(913000, 4)


,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [14]:
new_table = spark.read.format("delta").option("versionAsOf", 1).load("../lake")
print((new_table.count(), len(new_table.columns)))
new_table.limit(5).toPandas()

(913000, 5)


,date,store,item,sales,col_n
0,2013-01-01,1,1,13,2.0
1,2013-01-02,1,1,11,2.0
2,2013-01-03,1,1,14,2.0
3,2013-01-04,1,1,13,2.0
4,2013-01-05,1,1,10,2.0


In [22]:
# Appending new data. Steps OG Data v0 --> Added Column v1 --> Appending more data in V0 format
test = spark.read.option("header",True).csv('../data/test.csv')

test.count()

test.show(5)

45000

+---+----------+-----+----+
| id|      date|store|item|
+---+----------+-----+----+
|  0|2018-01-01|    1|   1|
|  1|2018-01-02|    1|   1|
|  2|2018-01-03|    1|   1|
|  3|2018-01-04|    1|   1|
|  4|2018-01-05|    1|   1|
+---+----------+-----+----+
only showing top 5 rows



In [23]:
test.write.format("delta").mode("append").save("../lake")

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 9609a4f6-8607-46cd-b1da-380d886da40d).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- date: string (nullable = true)
-- store: string (nullable = true)
-- item: string (nullable = true)
-- sales: string (nullable = true)
-- col_n: double (nullable = true)


Data schema:
root
-- id: string (nullable = true)
-- date: string (nullable = true)
-- store: string (nullable = true)
-- item: string (nullable = true)

         

In [32]:
# As seen above, schema is now as of v1. As v1 was same data as v0 but with additional columns we will try to delete v1 and revert to v0.
# For some reason following method deltaTable.restoreToVersion(0) is throwing an error, hence I will go for hacky version which is to load v0 in df and overwrite lake.

# Interestingly didn't have to use merge as of which i was expecting as v0 doesnt have new column

old_table.write.format("delta").mode("overwrite").save("../lake")

In [33]:
deltaTable.history().toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,2,2021-04-07 19:11:16.037,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,1.0,None,False,"{'numOutputRows': '913000', 'numOutputBytes': '1773875', 'numFiles': '4'}",None
1,1,2021-04-07 12:40:19.961,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,None,False,"{'numOutputRows': '913000', 'numOutputBytes': '1776421', 'numFiles': '4'}",None
2,0,2021-04-07 12:22:02.304,None,None,WRITE,"{'mode': 'ErrorIfExists', 'partitionBy': '[]'}",None,None,None,NaN,None,True,"{'numOutputRows': '913000', 'numOutputBytes': '1773875', 'numFiles': '4'}",None


In [34]:
df = spark.read.format("delta").load("../lake")
print((df.count(), len(df.columns)))
df.limit(5).toPandas()

(913000, 5)


,date,store,item,sales,col_n
0,2013-01-01,1,1,13,NaN
1,2013-01-02,1,1,11,NaN
2,2013-01-03,1,1,14,NaN
3,2013-01-04,1,1,13,NaN
4,2013-01-05,1,1,10,NaN


In [38]:
# Now we know why we didnt need merge, as the latest version simply nulled out col_n
df.select(F.countDistinct("col_n")).show()

+---------------------+
|count(DISTINCT col_n)|
+---------------------+
|                    0|
+---------------------+



In [40]:
# Lets try to write test table again and check history etc.
test.write.format("delta").mode("append").option("mergeSchema", "true").save("../lake")

deltaTable.history().toPandas()

df = spark.read.format("delta").load("../lake")
print((df.count(), len(df.columns)))
df.limit(5).toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,3,2021-04-07 19:17:32.460,None,None,WRITE,"{'mode': 'Append', 'partitionBy': '[]'}",None,None,None,2.0,None,True,"{'numOutputRows': '45000', 'numOutputBytes': '198755', 'numFiles': '1'}",None
1,2,2021-04-07 19:11:16.037,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,1.0,None,False,"{'numOutputRows': '913000', 'numOutputBytes': '1773875', 'numFiles': '4'}",None
2,1,2021-04-07 12:40:19.961,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,None,False,"{'numOutputRows': '913000', 'numOutputBytes': '1776421', 'numFiles': '4'}",None
3,0,2021-04-07 12:22:02.304,None,None,WRITE,"{'mode': 'ErrorIfExists', 'partitionBy': '[]'}",None,None,None,NaN,None,True,"{'numOutputRows': '913000', 'numOutputBytes': '1773875', 'numFiles': '4'}",None


(958000, 6)


,date,store,item,sales,col_n,id
0,2013-01-01,1,1,13,NaN,None
1,2013-01-02,1,1,11,NaN,None
2,2013-01-03,1,1,14,NaN,None
3,2013-01-04,1,1,13,NaN,None
4,2013-01-05,1,1,10,NaN,None


In [41]:
!head -n 5 ../data/train.csv

date,store,item,sales
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13


In [42]:
!head -n 5 ../data/test.csv

id,date,store,item
0,2018-01-01,1,1
1,2018-01-02,1,1
2,2018-01-03,1,1
3,2018-01-04,1,1
